# Tutorial demonstrating verification of v1 nino3 index against jra55

#### Import pyLatte package

In [1]:
from pylatte import utils
from pylatte import skill
from pylatte import indices

#### Currently, the following packages are required to load the data - this process will be replaced by the CAFE cookbook

In [2]:
import numpy as np
import pandas as pd
import xarray as xr
from dateutil.relativedelta import relativedelta

#### Import some plotting packages and widgets

In [3]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import warnings    
warnings.filterwarnings("ignore")

# Jupyter specific -----
from ipywidgets import FloatProgress
%matplotlib inline

# A note about the methodology of pyLatte
The pyLatte package is constructed around the xarray Python package. This is particularly useful for verification computations which require large numbers of samples (different model runs) to converge. 

The approach here is to generate very large xarray objects that reference all data required for the verification, but do not store the data in memory. Operations are performed on these xarray objects out-of-memory. When it is necessary to perform a compute (e.g. to produce a plot), this is distributed over multiple processors using the dask Python package.

# Initialise dask (currently not working on vm31)

In [4]:
# import dask
# import distributed
# client = distributed.Client(local_dir='/tmp/squ027-dask-worker-space', n_workers=4)
# client

# Construct xarray objects for forecasts and observations
(The CAFE cookbook will replace these code blocks)

In [5]:
# Location of forecast data -----
fcst_folder = '/OSM/CBR/OA_DCFP/data/model_output/CAFE/forecasts/v1/'
fcst_filename = 'ocean_daily*'
fcst_variable = 'sst'

# Location of observation data -----
obsv_folder = '/OSM/CBR/OA_DCFP/data/observations/sst/'
obsv_filename = 'HadISST.nc'
obsv_variable = 'sst'

In [6]:
# Initial dates to include (takes approximately 1 min 30 sec per date) -----
init_dates = pd.date_range('2003-1','2003-3' , freq='1MS')

# Ensembles to include -----
ensembles = range(1,12)

# Forecast length -----
FCST_LENGTH = 2 # years

In [7]:
# Resampling details -----
resample_freq = 'MS'

### Construct forecasts xarray object
Note, dask has a known bug that manifests when trying to concatentate data containing timedelta64 arrays (see https://github.com/pydata/xarray/issues/1952 for further details). For example, try to concatenate the following two Datasets:

`In : path = '/OSM/CBR/OA_DCFP/data/model_output/CAFE/forecasts/v1/yr2002/mn7/'`

`In : ens5 = xr.open_mfdataset(path + 'OUTPUT.5/atmos_daily*.nc', autoclose=True)`

`In : ens6 = xr.open_mfdataset(path + 'OUTPUT.6/atmos_daily*.nc', autoclose=True)`

`In : xr.concat([ens5, ens6],'ensemble')`

`Out : TypeError: invalid type promotion`

The error here is actually caused by the variables `average_DT` and `time_bounds`, which are timedelta64 arrays. However, I still do not fully unstand the bug: concatenation of `ens4` and `ens5`, for example, works fine, even though `ens4` also contains the timedelta64 variables `average_DT` and `time_bounds`. Regardless, because of this bug, it is not possible currently to create an xarray Dataset object containing all model variables. Instead, only the variable of interest (i.e. `fcst_variable` and `obsv_variable`) are retained in the concatenated xarray object.

In [8]:
# Instantiate progress bar -----
f = FloatProgress(min=0, max=len(init_dates)*len(ensembles), description='Loading...') 
display(f)

# Loop over initial dates -----
fcst_list = []
for init_date in init_dates:
    year = init_date.year
    month = init_date.month
    
    # Loop over ensembles -----
    ens_list = []
    for ensemble in ensembles:
        # Signal to increment the progress bar -----
        f.value += 1 
        
        # Stack ensembles into a list -----
        path = fcst_folder + '/yr' + str(year) + '/mn' + str(month) + \
               '/OUTPUT.' + str(ensemble) + '/' + fcst_filename + '.nc'
        dataset = xr.open_mfdataset(path, autoclose=True)[fcst_variable]
        ens_list.append(dataset.resample(time=resample_freq) \
                               .mean(dim='time'))
        
    # Concatenate ensembles -----
    ens_object = xr.concat(ens_list, dim='ensemble')
    ens_object['ensemble'] = ensembles
    
    # Stack concatenated ensembles into a list for each initial date -----                       
    fcst_list.append(utils.datetime_to_leadtime(ens_object))

# Keep track of the lead time for each initialization -----
n_lead_time = [len(x.lead_time) for x in fcst_list]

# Concatenate initial dates -----
da_fcst = xr.concat(fcst_list, dim='init_date')

# Rechunk for chunksizes of at least 1,000,000 elements -----
da_fcst = utils.prune(da_fcst.chunk(chunks={'ensemble' : len(da_fcst.ensemble), 
                                            'lead_time' : len(da_fcst.lead_time)}).squeeze())

FloatProgress(value=0.0, description='Loading...', max=33.0)

#### Truncate the forecast lead times at 2 years
The January and July forecasts are run for 5 years rather than 2 years. The xarray concatenation above can deal with this, but fills the shorter forecasts with nans for lead times longer than 2 years. Let's get rid of some of these nans by truncating the forecasts at the lead time corresponding to the longest 2 year forecast.

In [9]:
max_increments = FCST_LENGTH * 12
n_trunc = max([i for i in n_lead_time if i <= max_increments])
da_fcst = da_fcst.isel(lead_time=range(n_trunc))

### Construct observations xarray object

In [13]:
# Instantiate progress bar -----
f = FloatProgress(min=0, max=1, description='Loading...') 
display(f)

# Dates are referenced to 0000-01-01 - xr.open_mfdataset cannot deal with this -----
year_shift = 1970 
dataset = xr.open_dataset(obsv_folder + obsv_filename, autoclose=True, decode_times=False)[obsv_variable]
time_units = 'days since ' + str(year_shift) + '-01-01'

decoded_time = xr.coding.times.decode_cf_datetime(dataset.time,time_units)
shifted_time = np.array([np.datetime64(time - relativedelta(years=year_shift)).astype('datetime64[ns]') 
                         for time in decoded_time])
dataset.coords['time'] = ('time', shifted_time, {'long_name' : 'time', 'decoded_using' : time_units })

# Resample to monthly frequency -----
da_obsv = dataset.resample(time=resample_freq).mean(dim='time')

# Stack by initial date to match forecast structure -----
da_obsv = utils.stack_by_init_date(da_obsv,da_fcst.init_date.values,n_trunc)
f.value += 1

# Rechunk for chunksizes of at least 1,000,000 elements -----
da_obsv = utils.prune(da_obsv.chunk(chunks={'init_date' : len(da_obsv.init_date)}).squeeze())

FloatProgress(value=0.0, description='Loading...', max=1.0)

# Compute the nino3 index

##### Extract forecast and observation
Note, we `compute()` the xarray objects here to save time later on. Once dask is working, it will probably be most sensible to leave the objects uncomputed.

In [15]:
with utils.timer():
    da_fcst = da_fcst.compute()
    da_obsv = da_obsv.compute()

   Elapsed: 462.51069951057434 sec


##### Load climatology data
Various climatologies are/will be accessable using `utils.load_mean_climatology()`. Here we use a climatology computed over the full 55 year jra reanalysis

In [ ]:
jra_clim = utils.load_mean_climatology('jra_1958-2016', 'slp', freq='MS')

da_jra_clim = jra_clim.compute()

##### Compute anomaly data
Recall that the forecast and observation data are saved as functions of lead time and initial date. The function `utils.anomalize()` computes anomalies given data and a climatology which each have a datetime dimension `time`. Thus it is necessary to first convert from the lead time/initial date format to a datetime format, then compute the anomaly, the convert back to the lead time/initial date format.  

In [ ]:
anomalize = lambda data, clim: utils.datetime_to_leadtime(
                                   utils.anomalize(
                                       utils.leadtime_to_datetime(data),clim))

In [ ]:
da_fcst_anom = da_fcst #da_fcst.groupby('init_date').apply(anomalize, clim=da_jra_clim)

da_obsv_anom = da_obsv.groupby('init_date').apply(anomalize, clim=da_jra_clim)

In [ ]:
compute_soi(da_obsv_anom, std_dim='lead_time')

In [ ]:
da_fcst_Tahiti = utils.get_nearest_point(ds_fcst['slp'], lat_Tahiti, lon_Tahiti) * 100
da_fcst_Tahiti_anom = da_fcst_Tahiti.groupby('init_date').apply(anomalize, clim=da_clim_Tahiti)
da_fcst_Tahiti_std = da_fcst_Tahiti_anom.std(dim='lead_time')
da_fcst_Tahiti_stdzd = da_fcst_Tahiti_anom / da_fcst_Tahiti_std

da_fcst_Darwin = utils.get_nearest_point(ds_fcst['slp'], lat_Darwin, lon_Darwin) * 100
da_fcst_Darwin_anom = da_fcst_Darwin.groupby('init_date').apply(anomalize, clim=da_clim_Darwin)
da_fcst_Darwin_std = da_fcst_Darwin_anom.std(dim='lead_time')
da_fcst_Darwin_stdzd = da_fcst_Darwin_anom / da_fcst_Darwin_std

MSD_fcst = (da_fcst_Tahiti_stdzd - da_fcst_Darwin_stdzd).std(dim='lead_time')

da_SOI_fcst = ((da_fcst_Tahiti_stdzd - da_fcst_Darwin_stdzd) / MSD_fcst).compute()

In [ ]:
da_obsv_Tahiti = utils.get_nearest_point(ds_obsv['slp'], lat_Tahiti, lon_Tahiti)
da_obsv_Tahiti_anom = da_obsv_Tahiti.groupby('init_date').apply(anomalize, clim=da_clim_Tahiti)
da_obsv_Tahiti_std = da_obsv_Tahiti_anom.std(dim='lead_time')
da_obsv_Tahiti_stdzd = da_obsv_Tahiti_anom / da_obsv_Tahiti_std

da_obsv_Darwin = utils.get_nearest_point(ds_obsv['slp'], lat_Darwin, lon_Darwin)
da_obsv_Darwin_anom = da_obsv_Darwin.groupby('init_date').apply(anomalize, clim=da_clim_Darwin)
da_obsv_Darwin_std = da_obsv_Darwin_anom.std(dim='lead_time')
da_obsv_Darwin_stdzd = da_obsv_Darwin_anom / da_obsv_Darwin_std

MSD_obsv = (da_obsv_Tahiti_stdzd - da_obsv_Darwin_stdzd).std(dim='lead_time')

da_SOI_obsv = ((da_obsv_Tahiti_stdzd - da_obsv_Darwin_stdzd) / MSD_obsv).compute()

##### Compute persistence data
This requires repeating the data at the first lead time over all lead times. `utils.repeat_data()` allows us to do this

In [ ]:
da_SOI_pers = utils.repeat_data(da_SOI_obsv,'lead_time')

### Plot one initialization date

In [ ]:
fig1 = plt.figure(figsize=(10,5))

ax = fig1.add_axes([0.1, 0.1, 0.8, 0.8])
ax.grid()
ax.plot(da_SOI_fcst['lead_time'],da_SOI_fcst.isel(init_date=[0]).squeeze())
ax.plot(da_SOI_obsv['lead_time'],da_SOI_obsv.isel(init_date=[0]).squeeze(),'k-',linewidth=2)
ax.set_xlabel('lead time')
ax.set_ylabel('SOI [Pa]');

# Compute some skill metrics

## (Continuous) ranked probability score

In [ ]:
# Specify bins for computation of cdf -----
bins = np.linspace(0,5,50)

# Compute ranked probability score -----
rps = skill.compute_rps(da_SOI_fcst, da_SOI_obsv, bins=bins, indep_dims=None, ensemble_dim='ensemble') \
           .groupby('init_date.month').mean(dim='init_date')

In [ ]:
with utils.timer():
    fig1 = plt.figure(figsize=(8,4))

    ax = fig1.add_axes([0.1, 0.1, 0.8, 0.8])
    ax.grid()
    ax.imshow(rps.transpose(), extent=[1,24,1,2])
    ax.set_xlabel('Lead time [months]')
    ax.set_ylabel('Initialized month');

## Root mean squared error

In [ ]:
with utils.timer():
    rms_error = skill.compute_rms_error(da_SOI_fcst, da_SOI_obsv, 
                                        indep_dims=None, ensemble_dim='ensemble') \
                     .groupby('init_date.month').mean(dim='init_date')

In [ ]:
with utils.timer():
    fig1 = plt.figure(figsize=(8,4))

    ax = fig1.add_axes([0.1, 0.1, 0.8, 0.8])
    ax.grid()
    ax.imshow(rms_error, extent=[1,24,1,2])
    ax.set_xlabel('Lead time [months]')
    ax.set_ylabel('Initialized month');

In [ ]:
with utils.timer():
    rms_error = skill.compute_rms_error(da_SOI_pers, da_SOI_obsv, 
                                        indep_dims=None, ensemble_dim=None) \
                     .groupby('init_date.month').mean(dim='init_date')

In [ ]:
with utils.timer():
    fig1 = plt.figure(figsize=(8,4))

    ax = fig1.add_axes([0.1, 0.1, 0.8, 0.8])
    ax.grid()
    ax.imshow(rms_error, extent=[1,24,1,2])
    ax.set_xlabel('Lead time [months]')
    ax.set_ylabel('Initialized month');

# Close dask client

In [ ]:
# with utils.timer():
#     client.close()